In [1]:
!rm -rf ./logs/ 
%run Model.ipynb
%run dataset_loader.ipynb
import tensorflow as tf
import numpy
import sys
#from dataset_loader import load_dataset

test_no = 58

#tf.enable_eager_execution()
INPUT_WIDTH = 64
INPUT_HEIGHT = 64

INPUT_SIZE = 64

INPUT_CHANNELS = 1

NUM_CLASSES = 5

LEARNING_RATE = 0.0001  # Original value: 0.01
# DECAY_POINT = 2000
DECAY_RATE = 10



DROPOUT_RATE = 0

EPOCHS = 12000
GLOBAL_STEPS = 100
BATCH_SIZE = 256

NUM_PARRALLEL_THREADS = 10

GRAYSCALE=True
IS_FREQUENCY = False

RESTORE = True

train_sub_dir = '/data/image_format/training_data_15000/*/*'
val_sub_dir = '/data/image_format/validation_data/*/*'
test_sub_dir = '/data/image_format/test_data/*/*'

frequency_train_sub_dir = '/data/cogisen_frequency_data/training_data_15000_cogisen/*/*/*.csv'
frequency_val_sub_dir = '/data/cogisen_frequency_data/validation_data_1500_cogisen/*/*/*.csv'
frequency_test_sub_dir = '/data/cogisen_frequency_data/test_data_1500_cogisen/*/*/*.csv'



# train_sub_dir = '/data/image_format/small_dataset/train/*/*'
# val_sub_dir = '/data/image_format/small_dataset/val/*/*'
# test_sub_dir = '/data/image_format/small_dataset/test/*/*'

# frequency_train_sub_dir = '/data/cogisen_frequency_data/small_dataset/train/*/*/*.csv'
# frequency_val_sub_dir = '/data/cogisen_frequency_data/small_dataset/val/*/*/*.csv'
# frequency_test_sub_dir = '/data/cogisen_frequency_data/small_dataset/test/*/*/*.csv'






home_dir = os.path.dirname(os.path.realpath('__file__'))

train_dir = home_dir + train_sub_dir
val_dir = home_dir + val_sub_dir
test_dir = home_dir + test_sub_dir


frequency_train_dir = home_dir + frequency_train_sub_dir
frequency_val_dir = home_dir + frequency_val_sub_dir
frequency_test_dir = home_dir + frequency_test_sub_dir



training_dataset= load_dataset(img_dir= train_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

# logdir = './log/simple_model/frequency_data/grayscale'+str(experiment_no)
# weight_dir = './weights/simple_model/frequency_data/grayscale/'+str(experiment_no)+'/model'


logdir = home_dir+'/log/2_test/test/' + str(test_no)

weight_dir = home_dir+'/weights/test/' +str(test_no)

weights = weight_dir +'/model'
if not os.path.exists(weight_dir):
    os.makedirs(os.path.join(weight_dir))
    


/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'XLA_GPU']

In [3]:
get_available_gpus()

['/device:XLA_GPU:0']

In [4]:

handle = tf.placeholder(tf.string, shape=[],name="data_handle")

with tf.name_scope('iterators'):
    training_iterator = tf.data.Iterator.from_structure(training_dataset.output_types,training_dataset.output_shapes)
    training_init_op = training_iterator.make_initializer(training_dataset)
    

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


In [5]:
iterator = tf.data.Iterator.from_string_handle(
    handle, training_iterator.output_types,training_dataset.output_shapes)

next_training_element = iterator.get_next()



In [6]:
#with tf.device('/device:XLA_GPU:0'):
model = Simple_Model(next_training_element,input_height=INPUT_HEIGHT,input_width=INPUT_WIDTH, num_classes=NUM_CLASSES,input_channels=INPUT_CHANNELS,dropout_rate=DROPOUT_RATE,learning_rate=LEARNING_RATE)


[None, 3, 3, 16]


In [7]:

#train_sub_dir = '/data/image_format/training_data_1500/*/*'
val_dataset = load_dataset(img_dir= val_dir,frequency_dir=frequency_val_dir,minibatch_size=1500,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset,name='val_init_op')
 

In [ ]:

# if RESTORE:
#     tf.reset_default_graph()
#     graph = tf.get_default_graph()
#     saver = tf.train.import_meta_graph(meta_dir)




with tf.device('/device:XLA_GPU:0'):

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        
        sess.run(tf.global_variables_initializer())
        
     
        file_writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

        
        summary_operation = tf.summary.merge_all()
        
        saver = tf.train.Saver()
        saver.save(sess, weights)
        training_string_handle =  sess.run(training_iterator.string_handle())
        val_handle = sess.run(val_iterator.string_handle())
        numpy.set_printoptions(threshold=sys.maxsize)
        
        # Original value: 0.01
        for epoch in range(EPOCHS):
            
            sess.run(training_init_op,feed_dict={handle:training_string_handle})
            saver.save(sess, weights,global_step=GLOBAL_STEPS,write_meta_graph=False)
            
#             if epoch == DECAY_POINT:
#                 LEARNING_RATE = LEARNING_RATE/DECAY_RATE
                
            while True:
                
                try:
                    _, train_accuracy,summary,input_image,predictions= sess.run([model.training_operation,model.accuracy_operation,summary_operation,model.X,model.predictions],feed_dict={handle:training_string_handle,model.learning_rate:LEARNING_RATE})
            
                except tf.errors.OutOfRangeError:
                    
                    file_writer.add_summary(summary, epoch)
                    
                    if epoch%5 ==0:
                        print("accuracy for training set for epoch {0} is  :  {1}".format(epoch,train_accuracy))
                        #print("input_image = ",input_image[0][0])
                        print("predictions: ",predictions[:5])
           
                    break
            
            
            
            
            if epoch%5 ==0:
                sess.run(val_init_op,feed_dict={handle:val_handle})
            

                while True:
                    try:

                        val_accuracy,loss = sess.run([model.accuracy_operation,model.loss],feed_dict={handle:val_handle})
                        
                    except tf.errors.OutOfRangeError:
                        
                        print("accuracy for validation set for epoch {0} is  :  {1} and loss is {2}".format(epoch,val_accuracy,loss))

                        break


                
                


            
            

                
            
           





accuracy for training set for epoch 0 is  :  0.19078947603702545
predictions:  [[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
accuracy for validation set for epoch 0 is  :  0.2006666660308838 and loss is 5469652662616064.0
accuracy for training set for epoch 5 is  :  0.20394736528396606
predictions:  [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
accuracy for validation set for epoch 5 is  :  0.17466667294502258 and loss is 873599904776192.0
accuracy for training set for epoch 10 is  :  0.2763157784938812
predictions:  [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
accuracy for validation set for epoch 10 is  :  0.24199999868869781 and loss is 446991742009344.0
accuracy for training set for epoch 15 is  :  0.31578946113586426
predictions:  [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]
accuracy for validation set

accuracy for validation set for epoch 150 is  :  0.218666672706604 and loss is 61799139377152.0
accuracy for training set for epoch 155 is  :  0.25
predictions:  [[0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]
accuracy for validation set for epoch 155 is  :  0.21666666865348816 and loss is 60525018873856.0
accuracy for training set for epoch 160 is  :  0.19736842811107635
predictions:  [[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
accuracy for validation set for epoch 160 is  :  0.2160000056028366 and loss is 59231310970880.0
accuracy for training set for epoch 165 is  :  0.30263158679008484
predictions:  [[0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]
accuracy for validation set for epoch 165 is  :  0.21266666054725647 and loss is 58138325680128.0
accuracy for training set for epoch 170 is  :  0.23026315867900848
predictions:  [[1. 0. 0. 0. 0.]
 [0. 0. 0. 

accuracy for validation set for epoch 305 is  :  0.23066666722297668 and loss is 26703615229952.0
accuracy for training set for epoch 310 is  :  0.25
predictions:  [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
accuracy for validation set for epoch 310 is  :  0.2266666740179062 and loss is 26289979260928.0
accuracy for training set for epoch 315 is  :  0.28289473056793213
predictions:  [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
accuracy for validation set for epoch 315 is  :  0.22200000286102295 and loss is 25986796093440.0
accuracy for training set for epoch 320 is  :  0.25
predictions:  [[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
accuracy for validation set for epoch 320 is  :  0.22333332896232605 and loss is 25415921958912.0
accuracy for training set for epoch 325 is  :  0.25
predictions:  [[0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0

accuracy for validation set for epoch 460 is  :  0.20999999344348907 and loss is 18667609784320.0
accuracy for training set for epoch 465 is  :  0.21710526943206787
predictions:  [[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]
accuracy for validation set for epoch 465 is  :  0.20666666328907013 and loss is 18527805243392.0
accuracy for training set for epoch 470 is  :  0.29605263471603394
predictions:  [[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
accuracy for validation set for epoch 470 is  :  0.21066667139530182 and loss is 18433114636288.0
accuracy for training set for epoch 475 is  :  0.25
predictions:  [[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
accuracy for validation set for epoch 475 is  :  0.24400000274181366 and loss is 18293062631424.0
accuracy for training set for epoch 480 is  :  0.23026315867900848
predictions:  [[0. 0. 1. 0. 0.]
 [0. 0. 

accuracy for validation set for epoch 615 is  :  0.23466666042804718 and loss is 14903512399872.0
accuracy for training set for epoch 620 is  :  0.29605263471603394
predictions:  [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
accuracy for validation set for epoch 620 is  :  0.24199999868869781 and loss is 14808640389120.0
accuracy for training set for epoch 625 is  :  0.2631579041481018
predictions:  [[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
accuracy for validation set for epoch 625 is  :  0.23933333158493042 and loss is 14704765304832.0
accuracy for training set for epoch 630 is  :  0.34210526943206787
predictions:  [[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]
accuracy for validation set for epoch 630 is  :  0.23733332753181458 and loss is 14583095885824.0
accuracy for training set for epoch 635 is  :  0.25
predictions:  [[0. 0. 1. 0. 0.]
 [0. 0. 1

...

accuracy for validation set for epoch 5245 is  :  0.2473333328962326 and loss is 1935415312384.0
accuracy for training set for epoch 5250 is  :  0.375
predictions:  [[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]
accuracy for validation set for epoch 5250 is  :  0.23000000417232513 and loss is 1937260544000.0
accuracy for training set for epoch 5255 is  :  0.42763158679008484
predictions:  [[0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
accuracy for validation set for epoch 5255 is  :  0.2526666522026062 and loss is 1934075625472.0
accuracy for training set for epoch 5260 is  :  0.40789473056793213
predictions:  [[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]]
accuracy for validation set for epoch 5260 is  :  0.25 and loss is 1928988590080.0
accuracy for training set for epoch 5265 is  :  0.46052631735801697
predictions:  [[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [

accuracy for validation set for epoch 5400 is  :  0.24666666984558105 and loss is 1877229436928.0
accuracy for training set for epoch 5405 is  :  0.42105263471603394
predictions:  [[0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]
accuracy for validation set for epoch 5405 is  :  0.2266666740179062 and loss is 1875304644608.0
accuracy for training set for epoch 5410 is  :  0.43421053886413574
predictions:  [[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
accuracy for validation set for epoch 5410 is  :  0.2486666738986969 and loss is 1873502273536.0
accuracy for training set for epoch 5415 is  :  0.3486842215061188
predictions:  [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
accuracy for validation set for epoch 5415 is  :  0.25066667795181274 and loss is 1875746619392.0
accuracy for training set for epoch 5420 is  :  0.44078946113586426
predictions:  [[0. 0. 1.

accuracy for validation set for epoch 5555 is  :  0.25 and loss is 1823725453312.0
accuracy for training set for epoch 5560 is  :  0.43421053886413574
predictions:  [[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]
accuracy for validation set for epoch 5560 is  :  0.23399999737739563 and loss is 1830037487616.0
accuracy for training set for epoch 5565 is  :  0.3486842215061188
predictions:  [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
accuracy for validation set for epoch 5565 is  :  0.23133333027362823 and loss is 1825585889280.0
accuracy for training set for epoch 5570 is  :  0.3881579041481018
predictions:  [[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]
accuracy for validation set for epoch 5570 is  :  0.23333333432674408 and loss is 1823505121280.0
accuracy for training set for epoch 5575 is  :  0.375
predictions:  [[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [

accuracy for validation set for epoch 5710 is  :  0.2526666522026062 and loss is 1775085420544.0
accuracy for training set for epoch 5715 is  :  0.41447368264198303
predictions:  [[0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
accuracy for validation set for epoch 5715 is  :  0.25200000405311584 and loss is 1769867837440.0
accuracy for training set for epoch 5720 is  :  0.41447368264198303
predictions:  [[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]]
accuracy for validation set for epoch 5720 is  :  0.2486666738986969 and loss is 1766705332224.0
accuracy for training set for epoch 5725 is  :  0.3881579041481018
predictions:  [[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]
accuracy for validation set for epoch 5725 is  :  0.24400000274181366 and loss is 1771667062784.0
accuracy for training set for epoch 5730 is  :  0.46052631735801697
predictions:  [[0. 0. 0.

accuracy for validation set for epoch 5865 is  :  0.2473333328962326 and loss is 1724617850880.0
accuracy for training set for epoch 5870 is  :  0.4736842215061188
predictions:  [[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
accuracy for validation set for epoch 5870 is  :  0.24799999594688416 and loss is 1722503790592.0
accuracy for training set for epoch 5875 is  :  0.3684210479259491
predictions:  [[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]
accuracy for validation set for epoch 5875 is  :  0.24199999868869781 and loss is 1726519705600.0
accuracy for training set for epoch 5880 is  :  0.3552631437778473
predictions:  [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]]
accuracy for validation set for epoch 5880 is  :  0.2446666657924652 and loss is 1717072429056.0
accuracy for training set for epoch 5885 is  :  0.33552631735801697
predictions:  [[0. 0. 0. 1

In [ ]:
meta_dir = weights+'.meta'



In [ ]:
tf.reset_default_graph()
graph = tf.get_default_graph()
saver = tf.train.import_meta_graph(meta_dir)

val_dataset = load_dataset(img_dir= val_dir,frequency_dir=frequency_val_dir,minibatch_size=1500,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset,name='val_init_op')
 

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    
    print('validation dataset...')
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(weight_dir))
   
    iterator_handle = graph.get_tensor_by_name('data_handle:0')
    val_handle = sess.run(val_iterator.string_handle())
    

    
    sess.run(val_init_op,feed_dict={iterator_handle:val_handle})

    accuracy_op = graph.get_tensor_by_name('accuracy/accuracy_operation:0')
   
    while True:
        try:
            val_accuracy = sess.run([accuracy_op],feed_dict={iterator_handle:val_handle})

        except tf.errors.OutOfRangeError:
            print("accuracy for validation set :  {0}".format(val_accuracy))
            break



In [ ]:
tf.reset_default_graph()
graph = tf.get_default_graph()
saver = tf.train.import_meta_graph(meta_dir)

test_dataset = load_dataset(img_dir= test_dir,frequency_dir=frequency_test_dir,minibatch_size=1500,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,test_dataset.output_shapes)
test_init_op = test_iterator.make_initializer(test_dataset,name='test_init_op')


with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    
    print('testing dataset...')
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(weight_dir))
   # val_op = graph.get_operation_by_name("iterators/val_init_op")
    iterator_handle = graph.get_tensor_by_name('data_handle:0')
    test_handle = sess.run(test_iterator.string_handle())

    
    sess.run(test_init_op,feed_dict={iterator_handle:test_handle})

    accuracy_op = graph.get_tensor_by_name('accuracy/accuracy_operation:0')
   
    while True:
        try:
            test_accuracy = sess.run([accuracy_op],feed_dict={iterator_handle:test_handle})

        except tf.errors.OutOfRangeError:
            print("accuracy for testing set :  {0}".format(test_accuracy))
            break

